In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [38]:
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2 as l2

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from kerastuner import HyperModel

/var/folders/39/bf3hb7111_5fnn18n3p6z4rm0000gn/T/ipykernel_88588/3775580566.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [7]:
X_train = X_train / 255.0
X_val = X_val / 255.0

*Normal*

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/Users/pravinpb/pycode/MCW/Assignments/cifar10/cifar/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
model.fit(X_train, Y_train, epochs=20, batch_size=20, validation_data=(X_val, Y_val))

Epoch 1/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.8301 - loss: 0.4843 - val_accuracy: 0.6984 - val_loss: 1.0186
Epoch 2/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8393 - loss: 0.4498 - val_accuracy: 0.7043 - val_loss: 1.0351
Epoch 3/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8603 - loss: 0.4039 - val_accuracy: 0.6937 - val_loss: 1.0710
Epoch 4/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8704 - loss: 0.3658 - val_accuracy: 0.6950 - val_loss: 1.1066
Epoch 5/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8797 - loss: 0.3335 - val_accuracy: 0.6859 - val_loss: 1.1756
Epoch 6/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.8930 - loss: 0.2998 - val_accuracy: 0.6945 - val_loss: 1.2446
Epoch 7/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9028 - loss: 0.2769 - val_accuracy: 0.6932 - val_loss: 1.3037
Epoch 8/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.9099 - loss: 0

In [69]:
model.evaluate(x_test / 255.0, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6651 - loss: 2.1798


[2.2744457721710205, 0.6617000102996826]

In [70]:
model.save('cifar10_model_1.h5')

*Keras Tuner*

In [16]:
def model_builder(hp):
    model = Sequential()

    # First convolutional layer
    hp_filters_1 = hp.Int('filters_1', min_value=32, max_value=128, step=32)  # Tune filters
    hp_kernel_1 = hp.Choice('kernel_size_1', values=[3, 5])  # Tune kernel size
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])  # Tune activation function

    model.add(Conv2D(filters=hp_filters_1, 
                     kernel_size=(hp_kernel_1, hp_kernel_1), 
                     activation=hp_activation, 
                     input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))  # Static pooling layer

    # Second convolutional layer
    hp_filters_2 = hp.Int('filters_2', min_value=64, max_value=256, step=64)
    hp_kernel_2 = hp.Choice('kernel_size_2', values=[3, 5])

    model.add(Conv2D(filters=hp_filters_2, 
                     kernel_size=(hp_kernel_2, hp_kernel_2), 
                     activation=hp_activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third convolutional layer
    hp_filters_3 = hp.Int('filters_3', min_value=64, max_value=256, step=64)
    model.add(Conv2D(filters=hp_filters_3, 
                     kernel_size=(3, 3), 
                     activation=hp_activation))

    # Flatten the feature maps
    model.add(Flatten())

    # Dense layer with tunable units
    hp_dense_units = hp.Int('dense_units', min_value=64, max_value=512, step=64)
    model.add(Dense(units=hp_dense_units, activation=hp_activation))

    # Output layer for 10 classes
    model.add(Dense(10, activation='softmax'))

    # Learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [17]:
import keras_tuner as kt

tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',  # We want to optimize for validation accuracy
    max_epochs=10,  # Number of epochs to run for each trial
    factor=3,  # Controls the resource allocation between trials
    directory='hyperband_results',  # Directory to store results
    project_name='cifar10_tuning'
)

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [21]:
tuner.search(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Trial 30 Complete [00h 02m 47s]
val_accuracy: 0.6406999826431274

Best val_accuracy So Far: 0.7251999974250793
Total elapsed time: 01h 18m 54s


In [27]:
best_model = tuner.get_best_models(num_models=1)[0]

/Users/pravinpb/pycode/MCW/Assignments/cifar10/cifar/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/pravinpb/pycode/MCW/Assignments/cifar10/cifar/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [28]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [29]:
best_model.fit(X_train, Y_train, epochs=50, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 52s 41ms/step - accuracy: 0.8338 - loss: 0.4674 - val_accuracy: 0.7238 - val_loss: 0.9001
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 55s 44ms/step - accuracy: 0.8709 - loss: 0.3657 - val_accuracy: 0.7265 - val_loss: 0.9164
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 54s 43ms/step - accuracy: 0.8851 - loss: 0.3227 - val_accuracy: 0.7311 - val_loss: 0.9682
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 55s 44ms/step - accuracy: 0.9049 - loss: 0.2677 - val_accuracy: 0.7189 - val_loss: 1.1136


In [30]:
best_model.evaluate(x_test / 255.0, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7203 - loss: 1.1297


[1.135772705078125, 0.7166000008583069]

In [31]:
best_model.save('cifar10_model_2.h5')

*With Regularization and Normalization*

In [39]:
def model_builder_2(hp):
    model = Sequential()

    # Input layer with data augmentation
    model.add(tf.keras.layers.RandomFlip("horizontal"))
    model.add(tf.keras.layers.RandomRotation(0.2))

    # First convolutional block
    hp_filters_1 = hp.Int('filters_1', min_value=32, max_value=128, step=32)
    hp_kernel_1 = hp.Choice('kernel_size_1', values=[3, 5])
    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'elu'])

    model.add(Conv2D(filters=hp_filters_1, 
                     kernel_size=(hp_kernel_1, hp_kernel_1), 
                     activation=hp_activation, 
                     kernel_regularizer=l2(1e-4), 
                     input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_rate_1', min_value=0.2, max_value=0.5, step=0.1)))

    # Second convolutional block
    hp_filters_2 = hp.Int('filters_2', min_value=64, max_value=256, step=64)
    hp_kernel_2 = hp.Choice('kernel_size_2', values=[3, 5])

    model.add(Conv2D(filters=hp_filters_2, 
                     kernel_size=(hp_kernel_2, hp_kernel_2), 
                     activation=hp_activation, 
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_rate_2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten and Dense layers
    model.add(Flatten())
    hp_dense_units = hp.Int('dense_units', min_value=64, max_value=512, step=64)
    model.add(Dense(units=hp_dense_units, activation=hp_activation, kernel_regularizer=l2(1e-4)))
    model.add(Dropout(rate=hp.Float('dropout_rate_dense', min_value=0.2, max_value=0.5, step=0.1)))

    # Output layer
    model.add(Dense(10, activation='softmax'))

    # Compile model
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [40]:
import keras_tuner as kt

tuner = kt.Hyperband(
    model_builder_2,
    objective='val_accuracy',  # We want to optimize for validation accuracy
    max_epochs=10,  # Number of epochs to run for each trial
    factor=3,  # Controls the resource allocation between trials
    directory='hyperband_results_2',  # Directory to store results
    project_name='cifar10_tuning'
)

/Users/pravinpb/pycode/MCW/Assignments/cifar10/cifar/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [42]:
tuner.search(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Trial 30 Complete [00h 11m 06s]
val_accuracy: 0.45980000495910645

Best val_accuracy So Far: 0.6133999824523926
Total elapsed time: 03h 08m 07s


In [43]:
best_model_3 = tuner.get_best_models(num_models=1)[0]

/Users/pravinpb/pycode/MCW/Assignments/cifar10/cifar/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/pravinpb/pycode/MCW/Assignments/cifar10/cifar/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [45]:
best_model_3.fit(X_train, Y_train, epochs=50, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 186s 148ms/step - accuracy: 0.5623 - loss: 1.5247 - val_accuracy: 0.5757 - val_loss: 1.5009
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 178s 142ms/step - accuracy: 0.5730 - loss: 1.4927 - val_accuracy: 0.5987 - val_loss: 1.4333
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 174s 139ms/step - accuracy: 0.5829 - loss: 1.4728 - val_accuracy: 0.5361 - val_loss: 1.7105
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 175s 140ms/step - accuracy: 0.5924 - loss: 1.4524 - val_accuracy: 0.6369 - val_loss: 1.3695
Epoch 5/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 172s 137ms/step - accuracy: 0.5987 - loss: 1.4598 - val_accuracy: 0.6411 - val_loss: 1.3650
Epoch 6/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 174s 139ms/step - accuracy: 0.6007 - loss: 1.4476 - val_accuracy: 0.6357 - val_loss: 1.3400
Epoch 7/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 172s 138ms/step - accuracy: 0.6103 - loss: 1.4286 - val_accuracy: 0.6167 - val_loss: 1.4299
Epoch 8/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 171s 136ms/step - ac

In [46]:
best_model_3.evaluate(x_test / 255.0, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.6252 - loss: 1.3848


[1.385072112083435, 0.6263999938964844]

In [47]:
best_model_3.save('cifar10_model_3.h5')

*Data Augumentation*

In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using preprocessing Library

In [49]:
datagen = ImageDataGenerator(
    rotation_range=20,           
    width_shift_range=0.2,       
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,       
    fill_mode='nearest'         
)

In [50]:
datagen.fit(X_train)

In [56]:
best_model.fit(datagen.flow(X_train, Y_train, batch_size=64),
          epochs=50,
          validation_data=(X_val, Y_val))

Epoch 1/50


/Users/pravinpb/pycode/MCW/Assignments/cifar10/cifar/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 54s 85ms/step - accuracy: 0.5591 - loss: 1.3361 - val_accuracy: 0.7007 - val_loss: 0.8561
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 55s 88ms/step - accuracy: 0.6152 - loss: 1.1005 - val_accuracy: 0.7050 - val_loss: 0.8756
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 56s 89ms/step - accuracy: 0.6267 - loss: 1.0624 - val_accuracy: 0.7218 - val_loss: 0.8026
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 55s 87ms/step - accuracy: 0.6456 - loss: 1.0195 - val_accuracy: 0.7338 - val_loss: 0.7831
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 56s 89ms/step - accuracy: 0.6567 - loss: 0.9851 - val_accuracy: 0.7230 - val_loss: 0.8019
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 57s 91ms/step - accuracy: 0.6669 - loss: 0.9605 - val_accuracy: 0.7244 - val_loss: 0.8330
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 56s 89ms/step - accuracy: 0.6748 - loss: 0.9426 - val_accuracy: 0.7307 - val_loss: 0.7924
Epoch 8/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 58s 92ms/step - accuracy: 0.6767 - loss: 0.9225 - val_accurac

In [57]:
best_model.evaluate(x_test / 255.0, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.7979 - loss: 0.6310


[0.6364809274673462, 0.7967000007629395]

In [58]:
best_model.save('cifar10_model_4.h5')